# Setup

You need to run this before start to adjust the setup.ini file.

In [42]:
def update_setup(cwd_path: str) -> None:
    import configparser
    import os

    config = configparser.ConfigParser()
    config.sections()

    file_path = os.path.join('..','setup.ini')

    config.read(file_path)

    config["Paths"]["PUMLE_ROOT"] = cwd_path

    with open(file_path, 'w') as configfile:
        config.write(configfile)

CWD_PATH = "/".join(os.getcwd().split('/')[:-1])
GENERICAL_PATH = "/path/to/PUMLE"

# Simulation parameters

Notebook objectives are below. Later on, move from this blueprint version to pure Python scripts.

- create structure to define simulation input parameters
- export to MATLAB/Octave format for MRST run

In [43]:
import platform, os, configparser
from datetime import datetime


def get_params(c: configparser) -> dict:   
    """
    Get simulation parameters from configuration setup file.
    
    Parameters
    ----------
        c : configparser
        
    Returns
    -------
    
        Dictionary of parameters.
    
    """ 
    
    # Read input parameters
    c.read(os.path.join('..','setup.ini'))
    

    # Sections
    sections = ['Paths', 'Pre-Processing', 'Grid', 'Fluid', 'Initial Conditions', 
                        'Boundary Conditions', 'Wells', 'Schedule']
    
    # Auxiliary function to reduce coding. 
    # Here, 'section=section' is used to bypass late binding and capture the looped value.
    # Otherwise, the 'section' value at lambda definition time would be the last looped value.
    aux = [lambda param, section=section: c.get(section, param) for section in sections]

    # TODO Remove globals and implement class. Should we transfer that to a Makefile??
    
    # Paths 
    global PUMLE_ROOT;      PUMLE_ROOT = c.get('Paths','PUMLE_ROOT') 
    global PUMLE_RESULTS;   PUMLE_RESULTS = c.get('Paths','PUMLE_RESULTS') 
    
    p_params = ['PUMLE_ROOT', 'PUMLE_RESULTS']    
        
    # Pre-processing
    pp_params = ['case_name', 'file_basename', 'model_name']    
                                      
    # Grid
    gp_params = ['file_path', 'repair_flag']
    
    # Fluid
    fp_params = ['pres_ref', 'temp_ref', 'cp_rock', 'srw', 'src', 'pe', 'XNaCl', 'mu_brine']

    # Initial Conditions
    sp_params = ['sw_0']

    # TODO Study MRST::addBC
    # Boundary conditions
    bc_params = ['type']

    # Well
    w_params = ['CO2_inj']

    # Schedule
    s_params = ['injection_time', 'migration_time', 'injection_timestep_rampup', 'migration_timestep']

    
    # Fetch sections to return a dict whose keys are sections and values are second-level dicts of parameters
    all_params = [p_params, pp_params, gp_params, fp_params, sp_params, bc_params, w_params, s_params]
    
    PARAMS = {}
    for k in range(len(all_params)):
        PARAMS[sections[k]] = dict(zip(all_params[k], [aux[k](_) for _ in all_params[k]]))
        
    print(f'[PUMLE] Simulation setup file sucessfully read.')

    return PARAMS

def read_sim_params():
    """
    Read simulation parameters from the configuration file 'setup.ini'. 
    
    TODO Refactor this function to better define the top project folder and allow setup.ini to come from other path.

    Returns
    -------
    dict: 
    
    """
                    
    # Search in directory above
    if 'setup.ini' not in os.listdir('..'): 
        raise RuntimeError('File \'setup.ini\' not found in the top project folder. Change working directory and rerun this script.')
   
    else: 
        # Get path to top folder project inside 'setup.ini'                            
        c = configparser.ConfigParser()
        return get_params(c)
    
    

def dict_to_ini(config_dict):
    """Helper function to convert a dict back to .ini format"""
    
    ini_str = ""
    for section, items in config_dict.items():
        ini_str += f"[{section}]\n"
        for key, value in items.items():
            ini_str += f"{key} = {value}\n"
        ini_str += "\n"
    return ini_str



def print_report(PARAMS: dict, res_dir: str, msg: bool=True) -> None:
    """
    Print simulation setup report for log purposes.
    
    Parameters
    ---------
        PARAMS: dictionary of parameters
        res_dir: results output directory
        msg: log message
    
    """
    
    # Defaults results folder to '/temp'
    out = os.path.join(PUMLE_ROOT,res_dir)
    print(out)

    if len(res_dir) == 0:
        out = os.path.join(PUMLE_ROOT,'temp')
        os.makedirs(out,exist_ok=True)
    else:
        os.makedirs(out,exist_ok=True)
            
    # Get date/time and system information
    system, hostname, release, *_ =  platform.uname()
    date_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Text elements to write
    te = {
        0: ''.center(80,'-') + '\n',
        1: 'PUMLE SIMULATION REPORT'.center(80, ' ') + '\n',     
        2: f'Date/Time: {date_time}\n',   
        3: f'OS: {system}\n',
        4: f'Version: {release}\n',
        5: f'Hostname: {hostname}\n'
    }
    
    # Write report
    with open(os.path.join(PUMLE_ROOT,res_dir,'report.txt'),'w',) as fo:
        
        # Header
        fo.write(te[0])
        fo.write(te[1])
        fo.write(te[0])
                
        for k in range(2,6): fo.write(te[k])
        
        # Core information
        fo.write(te[0])        
        fo.write(dict_to_ini(PARAMS)) # TODO Should we change to another structure?        
    
    fo.close()
    
    if msg: print(f'[PUMLE] Report file saved to \'{out}\'.')

In [44]:
def export_to_matlab(PARAMS) -> None:
    """ Export dict of simulation parameters to Matlab to be read individually."""
    
    from scipy.io import savemat
    
    for k in PARAMS.keys():    
        basename = f'{k.replace('-','').replace(' ','')}ParamsPUMLE'
        mroot = os.path.join(PUMLE_ROOT,'m')
        fname = os.path.join(mroot,basename + '.mat')
        savemat(fname, PARAMS[k], appendmat=True)
        print(f'[PUMLE] Matlab file \'{basename + '.mat'}\' exported to \'{mroot}\'.')
    

### TODO Refactor to script

In [45]:
def main():
    update_setup(CWD_PATH)
    # Pipeline test
    PARAMS = read_sim_params() 
    print_report(PARAMS,PUMLE_RESULTS)
    export_to_matlab(PARAMS)
    update_setup(GENERICAL_PATH)

if __name__ == "main":
    main()
    
main()

[PUMLE] Simulation setup file sucessfully read.
/home/luiz/dev/tcc/PUMLE/sim
[PUMLE] Report file saved to '/home/luiz/dev/tcc/PUMLE/sim'.
[PUMLE] Matlab file 'PathsParamsPUMLE.mat' exported to '/home/luiz/dev/tcc/PUMLE/m'.
[PUMLE] Matlab file 'PreProcessingParamsPUMLE.mat' exported to '/home/luiz/dev/tcc/PUMLE/m'.
[PUMLE] Matlab file 'GridParamsPUMLE.mat' exported to '/home/luiz/dev/tcc/PUMLE/m'.
[PUMLE] Matlab file 'FluidParamsPUMLE.mat' exported to '/home/luiz/dev/tcc/PUMLE/m'.
[PUMLE] Matlab file 'InitialConditionsParamsPUMLE.mat' exported to '/home/luiz/dev/tcc/PUMLE/m'.
[PUMLE] Matlab file 'BoundaryConditionsParamsPUMLE.mat' exported to '/home/luiz/dev/tcc/PUMLE/m'.
[PUMLE] Matlab file 'WellsParamsPUMLE.mat' exported to '/home/luiz/dev/tcc/PUMLE/m'.
[PUMLE] Matlab file 'ScheduleParamsPUMLE.mat' exported to '/home/luiz/dev/tcc/PUMLE/m'.
